In [36]:
pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1279 sha256=fa7108e195d350a9a5b51488cee7cacefbf4cf5095b813abc1edfa706b0f392e
  Stored in directory: c:\users\sehyu\appdata\local\pip\cache\wheels\0a\9e\ba\20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [15]:
# from urllib.request import urlopen
# html = urlopen("https://www.frip.co.kr/daily/learning/all?orderBy=popular&page=1")  
# bsObject = BeautifulSoup(html, "html.parser")
# print(bsObject)

In [8]:
import requests
from bs4 import BeautifulSoup

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
url = "https://www.frip.co.kr/products/108923"
webpage = requests.get(url, headers=headers)
soup = BeautifulSoup(webpage.content, "html.parser").body.text
soup
# print(soup)

"If you're seeing this message, that means JavaScript has been disabled on your browser, please enable JS to make this app work."